In [ ]:
import mdtraj as md
import numpy as np
import nglview as nv
from rdkit import Chem
from rdkit.Chem import AllChem

from openmm import app
from openmm import unit
import simtk.openmm as mm
from simtk.openmm.app import Modeller, ForceField, PDBFile

from openmmforcefields.generators import SMIRNOFFTemplateGenerator

import openff.toolkit
from openff.toolkit.typing.engines.smirnoff import ForceField as openffForcefield
from openff.toolkit.topology import Molecule, Topology

## Extracting a Ligand from 1QKU PDB

In [ ]:
traj = md.load("../1_system_inspection/1QKU.pdb")
topology = traj.topology
print(traj)
print(topology)

In [ ]:
# Extract all three ligands
ligands = traj.atom_slice(topology.select("not water and not protein"))
print(ligands.topology)
# Extract a single ligand
ligand = ligands.atom_slice(ligands.topology.select("chainid == 0"))
view = nv.show_mdtraj(ligand)
view

In [ ]:
ligand.save_pdb("./ligand.pdb")

In [ ]:
ligand_pdb = Chem.rdmolfiles.MolFromPDBFile("./ligand.pdb")
ligand_pdb

Bond orders are incorrect. PDB files do not store information about bond orders for ligands.

## Fixing ligand bonds

In [ ]:
# Assing bond orders from smiles
template_mol = Chem.MolFromSmiles("C[C@]12CC[C@@H]3c4ccc(cc4CC[C@H]3[C@@H]1CC[C@@H]2O)O")
ligand_fixed = AllChem.AssignBondOrdersFromTemplate(template_mol, ligand_pdb)
ligand_fixed

In [ ]:
ligand_fixed.GetNumConformers()

In [ ]:
# Add Hydrogens
ligand_H = Chem.AddHs(ligand_fixed, addCoords=True)
view = nv.show_rdkit(ligand_H)
view

In [ ]:
# Save ligand to file so openff can use it
writer = Chem.SDWriter("./ligand.sdf")
writer.write(ligand_H)

## Ligand System Parametrized with OpenFF Parsley

First load the ligand

In [ ]:
ligand = Molecule.from_file("./ligand.sdf")
lig_topology = ligand.to_topology()
print("Number of conformers: {}".format(len(ligand.conformers)))
lig_positions = ligand.conformers[0]

### Load the forcefield

In [ ]:
forcefield = ForceField("amber99sb.xml", "tip3p.xml")
# Teach OpenMM about the ligand molecule and the Parsley force field
smirnoff = SMIRNOFFTemplateGenerator(forcefield="openff-1.3.0.offxml", molecules=[ligand])
forcefield.registerTemplateGenerator(smirnoff.generator)

### Model and solvate ligand

In [ ]:
modeller = Modeller(lig_topology.to_openmm(), lig_positions)

In [ ]:
geometric_center = modeller.positions.sum(axis=0)/modeller.positions.shape[0]

In [ ]:
modeller.positions = modeller.positions - geometric_center

In [ ]:
max_size = max(max((pos[i] for pos in modeller.positions))-min((pos[i] for pos in modeller.positions)) for i in range(3))
vectors = mm.Vec3(1.0, 0, 0), mm.Vec3(1.0/3.0, 2.0*np.sqrt(2.0)/3.0,0.0), mm.Vec3(-1.0/3.0, np.sqrt(2.0)/3.0, np.sqrt(6.0)/3.0)
box_vectors = [(max_size + 14*unit.angstroms)*v for v in vectors]

#modeller.addSolvent(forcefield, model='tip3p', boxVectors = box_vectors, neutralize=True)
modeller.addSolvent(forcefield, model='tip3p', padding=14*unit.angstroms, neutralize=True)

In [ ]:
n_waters = 0
for chain in modeller.topology.chains():
    for residue in chain.residues():
        if residue.name=='HOH':
            n_waters += 1

print('n_waters: {}'.format(n_waters))

In [ ]:
system = forcefield.createSystem(
    modeller.topology, 
    nonbondedMethod=app.NoCutoff
)

In [ ]:
system.getNumParticles()

In [ ]:
# Visualize ligand with water
topology = modeller.getTopology()
positions = modeller.getPositions()
PDBFile.writeFile(topology, positions, open('ligand_solvated.pdb', 'w'))

In [ ]:
view = nv.show_structure_file("ligand_solvated.pdb")
view.add_licorice(selection="(not protein)") # View water molecules
view